SQL adalah salah satu bahasa populer untuk pemrosesan dan analisis data. Spark mendukung SQL untuk memproses DataFrame. Dalam latihan ini kita akan menggunakan spark SQL tanpa database.


In [3]:
#%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=4e1a7b1aaa7b7c592e098290795a772bedec046ea1b8f6834711e70f62098956
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

Inisialisasi spark session untuk berinteraksi dengan Spark cluster.

In [5]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

Download dataset

In [6]:
!wget https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz

--2023-10-07 08:17:31--  https://github.com/urfie/SparkSQL-dengan-Hive/raw/main/datasets/application_record_header.csv.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz [following]
--2023-10-07 08:17:31--  https://raw.githubusercontent.com/urfie/SparkSQL-dengan-Hive/main/datasets/application_record_header.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3175443 (3.0M) [application/octet-stream]
Saving to: ‘application_record_header.csv.gz’

application_record_ 100%[===================>]   3.03M  --.-KB/s    in 0.09s

Load ke dataframe

In [7]:
df = spark.read.csv("application_record_header.csv.gz", header=True, inferSchema=True)

Sebelum menggunakan SQL, kita perlu membuat temporary table dari dataframe yang akan kita olah.

Gunakan fungsi `createOrReplaceTempView(nama_tabel)` pada dataframe tersebut.

In [8]:
df.createOrReplaceTempView("app_record")

Selanjutnya kita bisa menggunakan nama tabel yang sudah kita definisikan dalam SQL statement.

Untuk mengeksekusi SQL statement, kita gunakan fungsi `sql(sqlstatement)` pada spark session.

In [16]:
spark.sql("select count(*) from app_record").show()

+--------+
|count(1)|
+--------+
|  438557|
+--------+



In [17]:
spark.sql("select * from app_record limit 5").show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

Kita akan melakukan join salah satu kolom dengan data referensi dan melakukan agregasi. Sebelumnya kita buat dataframe referensi dan membuat temporary viewnya

In [9]:
mydata = (
    ('Lower secondary',1),
    ('Secondary / secondary special',2),
    ('Academic degree',3),
    ('Incomplete higher',4),
    ('Higher education',5))

ref_edu = spark.createDataFrame(mydata).toDF("NAME_EDUCATION_TYPE", "EDU_LEVEL")
ref_edu.createOrReplaceTempView("ref_edu")
spark.sql("select * from ref_edu").show()

+--------------------+---------+
| NAME_EDUCATION_TYPE|EDU_LEVEL|
+--------------------+---------+
|     Lower secondary|        1|
|Secondary / secon...|        2|
|     Academic degree|        3|
|   Incomplete higher|        4|
|    Higher education|        5|
+--------------------+---------+



Kita lakukan join, agregat, kemudian kita simpan ke tabel

In [10]:
spark.sql("""SELECT edu_level, count(1) as number_of_app FROM
              (SELECT ref_edu.EDU_LEVEL as edu_level
                FROM app_record LEFT JOIN ref_edu
                ON app_record.NAME_EDUCATION_TYPE=ref_edu.NAME_EDUCATION_TYPE)
             GROUP BY edu_level SORT BY edu_level""").write.saveAsTable(name="aggregated_edu", mode="overwrite")

Kita tampilkan hasilnya dengan menggunakan perintah `describe formatted`

In [13]:
spark.sql("describe formatted aggregated_edu").show(truncate = False)

+----------------------------+--------------------------------------------+-------+
|col_name                    |data_type                                   |comment|
+----------------------------+--------------------------------------------+-------+
|edu_level                   |bigint                                      |NULL   |
|number_of_app               |bigint                                      |NULL   |
|                            |                                            |       |
|# Detailed Table Information|                                            |       |
|Catalog                     |spark_catalog                               |       |
|Database                    |default                                     |       |
|Table                       |aggregated_edu                              |       |
|Created Time                |Sat Oct 07 08:17:51 UTC 2023                |       |
|Last Access                 |UNKNOWN                                     | 

In [15]:
%ls -l /content/spark-warehouse/aggregated_edu/

total 4
-rw-r--r-- 1 root root 824 Oct  7 08:17 part-00000-3255669c-389b-4c60-beaa-de0defd50b98-c000.snappy.parquet
-rw-r--r-- 1 root root   0 Oct  7 08:17 _SUCCESS
